In [30]:
import os
import json
import pandas as pd
from glob import glob
from tqdm import tqdm

In [31]:
# Configuration - You can modify these variables
data_dir = r"C:\Users\weiha\Desktop\benchmark_result"  # Base directory containing results
pipeline_list = ["supervised"]  # List of pipelines to analyze
task_name_list =["MotionSourceRecognition","BreathingDetection_Subset","Localization","FallDetection","ProximityRecognition","HumanActivityRecognition","HumanIdentification"]
# task_name_list = ["BreathingDetection_Subset"]  # List of tasks to analyze
model_name_list = ["mlp", "lstm", "resnet18", "transformer", "vit", "patchtst", "timesformer1d"] # List of models to analyze


# Initialize an empty list to store results
results_data = []

# Iterate through all combinations
for pipeline in pipeline_list:
    for task_name in task_name_list:
        for model_name in model_name_list:
            # Find all experiment folders for the current combination
            exp_pattern = os.path.join(data_dir, pipeline, task_name, model_name, "params_*")
            exp_folders = glob(exp_pattern)
            
            print(f"Found {len(exp_folders)} experiments for {pipeline}/{task_name}/{model_name}")
            
            # Process each experiment
            for exp_folder in tqdm(exp_folders, desc=f"Processing {pipeline}/{task_name}/{model_name}"):
                # Extract experiment ID
                exp_id = os.path.basename(exp_folder)
                
                # Define paths for config and results files
                config_filename = f"{model_name}_{task_name}_config.json"
                results_filename = f"{model_name}_{task_name}_results.json"
                
                config_path = os.path.join(exp_folder, config_filename)
                results_path = os.path.join(exp_folder, results_filename)
                
                # Skip if either file doesn't exist
                if not os.path.exists(config_path) or not os.path.exists(results_path):
                    print(f"Missing files for {exp_folder}, skipping")
                    continue
                
                # Read config file
                try:
                    with open(config_path, 'r') as f:
                        config_data = json.load(f)
                    
                    # Extract required fields from config
                    learning_rate = config_data.get('learning_rate')
                    weight_decay = config_data.get('weight_decay')
                    seed = config_data.get('seed')
                    batch_size = config_data.get('batch_size')
                    
                    # Skip if seed is not available (old version)
                    if seed is None:
                        print(f"Seed not available in {exp_folder}, skipping")
                        continue
                        
                except Exception as e:
                    print(f"Error reading config file {config_path}: {e}")
                    continue
                
                # Read results file
                try:
                    with open(results_path, 'r') as f:
                        results_data_json = json.load(f)
                except Exception as e:
                    print(f"Error reading results file {results_path}: {e}")
                    continue
                
                # Create a dictionary for the current experiment
                exp_result = {
                    'pipeline': pipeline,
                    'task_name': task_name,
                    'model_name': model_name,
                    'experiment_id': exp_id,
                    'learning_rate': learning_rate,
                    'weight_decay': weight_decay,
                    'batch_size':batch_size,
                    'seed': seed,
                }
                
                # Extract test results for all test sets
                for test_name, test_results in results_data_json.items():
                    if isinstance(test_results, dict):
                        for metric_name, metric_value in test_results.items():
                            column_name = f"{test_name}_{metric_name}"
                            exp_result[column_name] = metric_value
                
                # Add to results list
                results_data.append(exp_result)


results_df = pd.DataFrame(results_data)


for task in task_name_list:
    for model in model_name_list:
        task_model_df = results_df[(results_df['task_name'] == task) & (results_df['model_name'] == model)]
        if not task_model_df.empty:
            print(f"\n{task} - {model} (count: {len(task_model_df)})")
            
            # Find metrics columns
            metric_columns = [col for col in task_model_df.columns if any(col.endswith(m) for m in ['_loss', '_accuracy', '_f1'])]
            
            if metric_columns:
                avg_metrics = task_model_df[metric_columns].mean()
                for metric, value in avg_metrics.items():
                    print(f"  {metric}: {value:.4f}")

results_df = results_df[results_df["batch_size"] == 128].reset_index(drop=True)



Found 3 experiments for supervised/MotionSourceRecognition/mlp


Processing supervised/MotionSourceRecognition/mlp: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1350.97it/s]


Seed not available in C:\Users\weiha\Desktop\benchmark_result\supervised\MotionSourceRecognition\mlp\params_d0537c49, skipping
Found 3 experiments for supervised/MotionSourceRecognition/lstm


Processing supervised/MotionSourceRecognition/lstm: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1483.31it/s]


Seed not available in C:\Users\weiha\Desktop\benchmark_result\supervised\MotionSourceRecognition\lstm\params_c579fb8f, skipping
Found 3 experiments for supervised/MotionSourceRecognition/resnet18


...

Processing supervised/HumanIdentification/patchtst: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1484.53it/s]


Seed not available in C:\Users\weiha\Desktop\benchmark_result\supervised\HumanIdentification\patchtst\params_6102ffcc, skipping
Found 3 experiments for supervised/HumanIdentification/timesformer1d


Processing supervised/HumanIdentification/timesformer1d: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1260.94it/s]

Seed not available in C:\Users\weiha\Desktop\benchmark_result\supervised\HumanIdentification\timesformer1d\params_bf0441f6, skipping

MotionSourceRecognition - mlp (count: 2)
  test_hard_loss: 0.0576
  test_hard_accuracy: 0.9776
  test_medium_loss: 0.0283
  test_medium_accuracy: 0.9916
  test_loss: 0.0340
  test_accuracy: 0.9882
  test_easy_loss: 0.0285
  test_easy_accuracy: 0.9859
  test_medium_id_loss: nan
  test_medium_id_accuracy: nan
  test_easy_id_loss: nan
  test_easy_id_accuracy: nan
  test_hard_id_loss: nan
  test_hard_id_accuracy: nan
  test_cross_user_loss: nan
  test_cross_user_accuracy: nan
  test_cross_env_loss: nan
  test_cross_env_accuracy: nan
  test_cross_device_loss: nan
  test_cross_device_accuracy: nan

MotionSourceRecognition - lstm (count: 2)
  test_hard_loss: 0.1056
  test_hard_accuracy: 0.9748
  test_medium_loss: 0.0563
  test_medium_accuracy: 0.9858
  test_loss: 0.0661
  test_accuracy: 0.9833
  test_easy_loss: 0.0597
  test_easy_accuracy: 0.9826
  test_medium_

  test_hard_loss: nan
  test_hard_accuracy: nan
  test_medium_loss: nan
  test_medium_accuracy: nan
  test_loss: 0.0002
  test_accuracy: 1.0000
  test_easy_loss: nan
  test_easy_accuracy: nan
  test_medium_id_loss: 0.0002
  test_medium_id_accuracy: 1.0000
  test_easy_id_loss: 0.0003
  test_easy_id_accuracy: 1.0000
  test_hard_id_loss: 0.0003
  test_hard_id_accuracy: 1.0000
  test_cross_user_loss: nan
  test_cross_user_accuracy: nan
  test_cross_env_loss: nan
  test_cross_env_accuracy: nan
  test_cross_device_loss: nan
  test_cross_device_accuracy: nan

Localization - transformer (count: 1)
  test_hard_loss: nan
  test_hard_accuracy: nan
  test_medium_loss: nan
  test_medium_accuracy: nan
  test_loss: 0.0524
  test_accuracy: 0.9897
  test_easy_loss: nan
  test_easy_accuracy: nan
  test_medium_id_loss: 0.0004
  test_medium_id_accuracy: 1.0000
  test_easy_id_loss: 0.0125
  test_easy_id_accuracy: 0.9972
  test_hard_id_loss: 0.1028
  test_hard_id_accuracy: 0.9799
  test_cross_user_loss: nan

In [33]:
# results_df[["task_name","model_name","test_accuracy", "test_f1_score"]]
results_df[results_df['task_name'] == 'BreathingDetection_Subset']

,pipeline,task_name,model_name,experiment_id,learning_rate,weight_decay,batch_size,seed,test_hard_loss,test_hard_accuracy,...,test_hard_id_f1_score,test_cross_user_loss,test_cross_user_accuracy,test_cross_user_f1_score,test_cross_env_loss,test_cross_env_accuracy,test_cross_env_f1_score,test_cross_device_loss,test_cross_device_accuracy,test_cross_device_f1_score
14,supervised,BreathingDetection_Subset,mlp,params_b6406f90,0.001,0.00001,128,42,NaN,NaN,...,0.965443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,supervised,BreathingDetection_Subset,lstm,params_223749b3,0.001,0.00001,128,42,NaN,NaN,...,0.983013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,supervised,BreathingDetection_Subset,resnet18,params_b745243f,0.001,0.00001,128,42,NaN,NaN,...,0.981257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,supervised,BreathingDetection_Subset,transformer,params_2a909648,0.001,0.00001,128,42,NaN,NaN,...,0.974209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,supervised,BreathingDetection_Subset,vit,params_71ea36b7,0.001,0.00001,128,42,NaN,NaN,...,0.990822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,supervised,BreathingDetection_Subset,patchtst,params_9716e813,0.001,0.00001,128,42,NaN,NaN,...,0.985160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,supervised,BreathingDetection_Subset,timesformer1d,params_dda68b66,0.001,0.00001,128,42,NaN,NaN,...,0.987696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Save results to CSV
output_path = "all_results_summary.csv"
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to {output_path}")

,pipeline,task_name,model_name,experiment_id,learning_rate,weight_decay,batch_size,seed,test_hard_loss,test_hard_accuracy,...,test_hard_id_f1_score,test_cross_user_loss,test_cross_user_accuracy,test_cross_user_f1_score,test_cross_env_loss,test_cross_env_accuracy,test_cross_env_f1_score,test_cross_device_loss,test_cross_device_accuracy,test_cross_device_f1_score
0,supervised,MotionSourceRecognition,mlp,params_54de1ed4,0.001,0.00001,128,11,0.054798,0.979658,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,supervised,MotionSourceRecognition,mlp,params_a5f8d464,0.001,0.00001,128,3,0.060461,0.975590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,supervised,MotionSourceRecognition,lstm,params_77e9c041,0.001,0.00001,128,11,0.110225,0.974776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,supervised,MotionSourceRecognition,lstm,params_e7d692b4,0.001,0.00001,128,3,0.100947,0.974776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,supervised,MotionSourceRecognition,resnet18,params_b826dac9,0.001,0.00001,128,11,0.018879,0.995118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,supervised,HumanIdentification,vit,params_e1e08ac2,0.001,0.00001,128,42,NaN,NaN,...,NaN,7.557310,0.0,0.0,3.855845,0.428825,0.600248,4.416115,0.363209,0.347642
75,supervised,HumanIdentification,patchtst,params_0604ae60,0.001,0.00001,256,42,NaN,NaN,...,NaN,7.165012,0.0,0.0,5.692275,0.054932,0.104143,4.587672,0.166528,0.102308
76,supervised,HumanIdentification,patchtst,params_ed591e95,0.001,0.00001,128,42,NaN,NaN,...,NaN,8.295083,0.0,0.0,6.046113,0.253101,0.403959,5.375069,0.283541,0.279678
77,supervised,HumanIdentification,timesformer1d,params_02b8a740,0.001,0.00001,128,42,NaN,NaN,...,NaN,9.847098,0.0,0.0,7.402718,0.153574,0.266257,6.580354,0.201383,0.163201


,pipeline,task_name,model_name,experiment_id,learning_rate,weight_decay,seed,test_hard_loss,test_hard_accuracy,test_hard_f1_score,...,test_hard_id_f1_score,test_cross_user_loss,test_cross_user_accuracy,test_cross_user_f1_score,test_cross_env_loss,test_cross_env_accuracy,test_cross_env_f1_score,test_cross_device_loss,test_cross_device_accuracy,test_cross_device_f1_score
66,supervised,HumanIdentification,mlp,params_4747a425,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,15.015316,0.0,0.0,6.160070,0.244241,0.392594,5.348451,0.398340,0.378278
67,supervised,HumanIdentification,mlp,params_9e1e3cfe,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,31.071847,0.0,0.0,21.896978,0.173361,0.295495,12.609159,0.350207,0.307947
68,supervised,HumanIdentification,lstm,params_7a6c1ec1,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,14.178662,0.0,0.0,5.814986,0.327820,0.493772,5.126755,0.427386,0.389160
69,supervised,HumanIdentification,lstm,params_f0bcc75e,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,11.537951,0.0,0.0,4.904699,0.373597,0.543969,3.999741,0.483817,0.439365
70,supervised,HumanIdentification,resnet18,params_be1d0e76,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,7.585659,0.0,0.0,4.306992,0.285588,0.444291,3.932239,0.317289,0.287162
71,supervised,HumanIdentification,resnet18,params_e8d8127d,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,8.858137,0.0,0.0,4.126777,0.425281,0.596768,3.757309,0.478562,0.451269
72,supervised,HumanIdentification,transformer,params_ddcd54fc,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,13.976666,0.0,0.0,7.880250,0.286178,0.445006,6.278616,0.394744,0.352011
73,supervised,HumanIdentification,vit,params_d7db3cf0,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,6.839838,0.0,0.0,4.999186,0.255464,0.406963,4.087926,0.327248,0.294357
74,supervised,HumanIdentification,vit,params_e1e08ac2,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,7.557310,0.0,0.0,3.855845,0.428825,0.600248,4.416115,0.363209,0.347642
75,supervised,HumanIdentification,patchtst,params_0604ae60,0.001,0.00001,42,NaN,NaN,NaN,...,NaN,7.165012,0.0,0.0,5.692275,0.054932,0.104143,4.587672,0.166528,0.102308
